In [2]:
import cv2
import log_reg
import numpy as np
from glob import glob
from matplotlib.pyplot import imread, imshow
from util import read_X_Y, get_data_col, filter_data_by_colval
from keras.models import Sequential 
from keras.layers import Dense, Activation, Reshape
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#TensorBoard
from keras.callbacks import TensorBoard
from time import time

In [2]:
file_names = sorted(glob('all-mias/*.pgm'))

In [59]:
headers = ['file_name', 'character', 'class', 'severity', 'x', 'y', 'radius']
filtered_X, Y_headers,Y_names = read_X_Y('dataset/data.txt',headers)

In [60]:
image_stack = np.array([imread(file_name) for file_name in filtered_X])

In [64]:
def produce_y_mask(Y,Y_names):
    for i in range(len(Y)):
        image = np.zeros((1024,1024,1))
        cv2.circle(image, (int(Y[i][0]),1024-int(Y[i][1])),int(Y[i][2]),(255),-1)
        name = 'mias_y_masked/' + Y_names[i] + '.png'
        cv2.imwrite(name, image)

In [65]:
# print(Y_headers)
produce_y_mask(Y_headers,Y_names)

In [66]:
Y_mask_filenames = ['mias_y_masked/' + file_name + '.png' for file_name in Y_names]
Y = np.array([imread(file_name) for file_name in Y_mask_filenames])

In [71]:
Y = np.array(Y)
Y = np.squeeze(Y)
m,l,h = image_stack.shape

print(image_stack.shape)
X = image_stack.reshape((m,l,h,1))
Y = Y.reshape((m,l*h,1))
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.6,random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_test,Y_test,train_size=0.5,random_state=42)

# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
# X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

# Y_train = Y_train.reshape((m,l*h,1))



(119, 1024, 1024)


/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(71, 1024, 1024, 1)
(71, 1048576, 1)
(24, 1024, 1024, 1)
(24, 1048576, 1)
(24, 1024, 1024, 1)
(24, 1048576, 1)


In [72]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_val = to_categorical(Y_val)

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(l,h,1)))
model.add(Conv2D(128, kernel_size=(5, 5),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(100, 100)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(5, 5),
                 activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1048576*2))
model.add(Reshape((1048576, 2)))
model.add(Activation('softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy')
# model.summary()
# print(X_train.shape)
# print(Y_train.shape)
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=32,verbose=1, callbacks=[tensorboard])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 1020, 1020, 64)    1664      
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 1016, 1016, 128)   204928    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 6, 6, 128)         409728    
_________________________________________________________________
flatten_6 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                294976    
__________

In [7]:
#General model seems to need an extremely large output!
model = Sequential([
    Dense(128, input_shape=(l*h,)),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(3),
    Activation('relu'),
])
model.compile(optimizer='Adam',
              loss='categorical_crossentropy')
model.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=32)

Train on 71 samples, validate on 24 samples
Epoch 1/20
71/71 [==============================] - 26s 366ms/step - loss: 16666.3377 - val_loss: 11683.5654
Epoch 2/20
71/71 [==============================] - 13s 185ms/step - loss: 7853.4531 - val_loss: 453.2060
Epoch 3/20
71/71 [==============================] - 11s 157ms/step - loss: 385.5853 - val_loss: 344.5707
Epoch 4/20
71/71 [==============================] - 8s 118ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 5/20
71/71 [==============================] - 10s 134ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 6/20
71/71 [==============================] - 13s 183ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 7/20
71/71 [==============================] - 10s 136ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 8/20
71/71 [==============================] - 9s 132ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 9/20
71/71 [==============================] - 10s 137ms/step - loss: 350.8421 - val_loss: 344.5707
Epoch 10

In [8]:
y_pred_val = model.predict(X_train)

In [9]:
print(y_pred_val)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [28]:
print(Y_val)

[[448. 480.  95.]
 [492. 473. 131.]
 [600. 514.  67.]
 [518. 191.  39.]
 [356. 945.  72.]
 [647. 503.  87.]
 [523. 482.  29.]
 [547. 520.  45.]
 [366. 620.  33.]
 [525. 425.  33.]
 [489. 480.  82.]
 [415. 460.  38.]
 [326. 607. 174.]
 [352. 624. 114.]
 [462. 406.  44.]
 [603. 538.  44.]
 [522. 553.  17.]
 [519. 362.  54.]
 [592. 670.  33.]
 [493. 125.  49.]
 [612. 297.  34.]
 [191. 549.  23.]
 [400. 484.  37.]
 [653. 477.  49.]]


In [ ]:
model2 = Sequential([
    Dense(1024, input_shape=(l*h,)),
    Activation('relu'),
    Dense(512),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(3),
    Activation('relu'),
])
model2.compile(optimizer='Adam',
              loss='mean_squared_error')
model2.fit(X_train,Y_train,epochs=20, validation_data=(X_val, Y_val), batch_size=32)

Train on 71 samples, validate on 24 samples
Epoch 1/20


In [18]:
model2 = Sequential([
    Dense(128, input_shape=(l*h,)),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(16),
    Activation('relu'),
    Dense(3),
    Activation('relu'),
])
model2.compile(optimizer='Adam',
              loss='mean_squared_error')
# print(X_train[0].shape)
model2.fit(,epochs=5, batch_size=32)

(1048576,)


ValueError: Error when checking target: expected activation_55 to have shape (3,) but got array with shape (1,)

In [9]:
image = Image.new('1', (1024, 1024)) #create new image, 10x10 pixels, 1 bit per pixel
print(Y_train[0])

[492. 434.  87.]


In [ ]:
draw = ImageDraw.Draw(image)
draw.ellipse((2, 2, 8, 8), outline ='white')
print list(image.getdata())